In [1]:
import minsearch
from openai import OpenAI
import json

In [2]:
with open('../documents.json', 'rt') as f:
    docs_raw = json.load(f)

In [3]:
documents = []

for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [4]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [5]:
client = OpenAI()

In [6]:
def search(query, index=index, filter_dict = 'data-engineering-zoomcamp'):
    boost = {'question': 3.0, 'section':0.5}
    data = index.search(
        query = query,
        boost_dict=boost,
        num_results=5,
        filter_dict={'course': filter_dict }
    )
    return data

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teacher assistant. Answer the question based on the context.
        Use only the facts from the context. If relevant information is missing, say you do not know.
        Question:
        {query}
        Context:
        {context}
        """.strip()
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
    return prompt_template.format(query=query, context=context).strip()

In [8]:
def llm(client, prompt, model='gpt-4.1-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt }]
    )
    return response.choices[0].message.content

In [17]:
def rag(query, verbose_search= False, verbose_prompt = False):
    search_results = search(q, index)
    if verbose_search:
        print(search_results)
    prompt = build_prompt(q,search_results)
    if verbose_prompt:
        print(prompt)
    message = llm(client, prompt)
    return(message)

In [19]:
q = "Give me a sample dockerfile"
rag(q, verbose_prompt=True)

You're a course teacher assistant. Answer the question based on the context.
        Use only the facts from the context. If relevant information is missing, say you do not know.
        Question:
        Give me a sample dockerfile
        Context:
        section: Module 1: Docker and Terraform
question: Docker - failed to solve with frontend dockerfile.v0: failed to read dockerfile: error from sender: open ny_taxi_postgres_data: permission denied.
answer:This happens on Ubuntu/Linux systems when trying to run the command to build the Docker container again.
$ docker build -t taxi_ingest:v001 .
A folder is created to host the Docker files. When the build command is executed again to rebuild the pipeline or create a new one the error is raised as there are no permissions on this new folder. Grant permissions by running this comtionmand;
$ sudo chmod -R 755 ny_taxi_postgres_data
Or use 777 if you still see problems. 755 grants write access to only the owner.

section: Module 4: analyti

'The context does not provide a sample Dockerfile.  \nTherefore, I do not know a sample Dockerfile based on the given information.'